In [0]:
# !pip install simpletransformers

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, accuracy_score
from simpletransformers.classification import ClassificationModel, MultiLabelClassificationModel
from multiprocessing import cpu_count
tqdm.pandas()

In [0]:
# read the csv file
df = pd.read_csv("drive/My Drive/df.csv", usecols=['text', 'link_flair_text'])

In [0]:
# encode the categorical flair into numerical labels
le = LabelEncoder().fit(df.link_flair_text)
df['label'] = le.transform(df.link_flair_text)

In [0]:
# split the dataset into train/test set
train, test = train_test_split(df[['text', 'label']], test_size=0.1) 

In [5]:
print(train.shape, test.shape)

(90334, 2) (10038, 2)


In [6]:
train.head()

,text,label
51462,Anyone interested in sharing a netlfix account...,0
10737,"'मैं एंटी- मोदी नहीं हूं, मैं बस उनसे सवाल पूछ...",7
66518,Somewhere I found in Auto Rickshaw redd flrytj,0
8452,Dish TV is giving me all sort of shit channels...,0
77093,"Bangalore to Bangkok by road. Hey guys, Me and...",0


In [0]:
args = {
    "output_dir": "outputs-xlnet/",
    "cache_dir": "cache-xlnet/",
    "best_model_dir": "outputs-xlnet/best_model/",

    "fp16": False,
    "fp16_opt_level": "O1",
    "max_seq_length": 128,
    "train_batch_size": 32,
    "eval_batch_size": 32,
    "gradient_accumulation_steps": 1,
    "num_train_epochs": 3,
    "weight_decay": 0,
    "learning_rate": 1e-5,
    "adam_epsilon": 1e-8,
    "warmup_ratio": 0.06,
    "warmup_steps": 0,
    "max_grad_norm": 1.0,
    "do_lower_case": False,

    "logging_steps": 50,
    "evaluate_during_training": False,
    "evaluate_during_training_steps": 2000,
    "evaluate_during_training_verbose": False,
    "use_cached_eval_features": False,
    "save_eval_checkpoints": True,
    "no_cache": False,
    "save_model_every_epoch": True,
    "tensorboard_dir": None,

    "overwrite_output_dir": True,
    "reprocess_input_data": True,

    "process_count": cpu_count() ,
    "n_gpu": 1,
    "silent": False,
    "use_multiprocessing": True,

    "wandb_project": None,
    "wandb_kwargs": {},

    "use_early_stopping": True,
    "early_stopping_patience": 2,
    "early_stopping_delta": 0,
    "early_stopping_metric": "eval_loss",
    "early_stopping_metric_minimize": True,

    "manual_seed": 0,
    "encoding": None,
    "config": {},
}

In [0]:
def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='micro')

In [0]:
def show_performance(model, train=train, test=test):
  print(f"Training Report after first 3 epochs")
  result, _, _ = model.eval_model(train, f1=f1_multiclass, acc=accuracy_score)
  print(result)
  print("Testing Report")
  result, _, _ = model.eval_model(test, f1=f1_multiclass, acc=accuracy_score)
  print(result)

#### DistilBERT(by Huggingface)
- Ref: https://github.com/ThilinaRajapakse/simpletransformers

In [0]:
# default args
model = ClassificationModel('distilbert', 'distilbert-base-cased', num_labels=11, args=args)

ValueError: ignored

In [0]:
# model.train_model(train_df=train)

In [0]:
model.train_model(train_df=train)

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:243: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


Running loss: 0.143318

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:224: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Running loss: 0.428641


Running loss: 0.348258


Running loss: 0.262035


Running loss: 0.037119


Running loss: 0.024125



In [0]:
print(f"Training Report after first {args['']}")
result, _, _ = model.eval_model(train, f1=f1_multiclass, acc=accuracy_score)
print(result)
print("Testing Report")
result, _, _ = model.eval_model(test, f1=f1_multiclass, acc=accuracy_score)
print(result)

Training Report


/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:660: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."



{'mcc': 0.9982554661220467, 'f1': 0.9985830362875551, 'acc': 0.9985830362875551, 'eval_loss': 0.006345467465265493}


In [0]:
print("Testing Report")
result, _, _ = model.eval_model(test, f1=f1_multiclass, acc=accuracy_score)
print(result)

Training Report


/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:660: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."



{'mcc': 0.5522069312804754, 'f1': 0.6396692568240685, 'acc': 0.6396692568240685, 'eval_loss': 2.347384988507138}


##### Training Time: ~20m//epoch

In [0]:
#zip the outputs and download the zippped
# !zip -r outputs-distil.zip outputs/

## XLNet (current SOTA for text classification)

In [0]:
# default args
model_xlnet = ClassificationModel('xlnet', 'xlnet-base-cased', num_labels=11, args=args)

In [11]:
model_xlnet.train_model(train_df=train)

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:247: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1340: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Running loss: 1.957498

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:224: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Running loss: 0.915517


Running loss: 0.942770



In [17]:
show_performance(model_xlnet, train, test)

Training Report after first 3 epochs


/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:664: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1340: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")



{'mcc': 0.6725041889240168, 'f1': 0.7346071246706667, 'acc': 0.7346071246706667, 'eval_loss': 0.7942420120489246}
Testing Report



{'mcc': 0.5668271435381923, 'f1': 0.6502291293086272, 'acc': 0.6502291293086272, 'eval_loss': 1.021399719320285}


In [21]:
model_xlnet.train_model(train_df=train)

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:247: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1340: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Running loss: 0.583949

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:224: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Running loss: 1.030137


Running loss: 1.038652



In [22]:
show_performance(model_xlnet, train, test)

Training Report after first 3 epochs


/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:664: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1340: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")



{'mcc': 0.7481896500420141, 'f1': 0.7952487435517082, 'acc': 0.7952487435517082, 'eval_loss': 0.6341553909682316}
Testing Report



{'mcc': 0.5589317036237982, 'f1': 0.6433552500498108, 'acc': 0.6433552500498108, 'eval_loss': 1.0854253151993842}


In [0]:
# lwoering the lr to 1e-5 and training for few 3 more epochs 
model_xlnet.train_model(train_df=train)

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:247: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1340: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Running loss: 0.529136

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:224: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Running loss: 0.907650


Running loss: 0.425779

In [0]:
!cp -r outputs-xlnet/ drive/My\ Drive/

In [4]:
!nvidia-smi

Fri Apr 24 21:43:56 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
model_xlnet.train_model(train_df=train)

In [22]:
!pip install numba

In [0]:
from numba import cuda
cuda.select_device(0)
cuda.close()

In [17]:
get_available_gpus()

['/device:GPU:0']

In [25]:
# setting device on GPU if available, else CPU
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')

Using device: cuda

Tesla P100-PCIE-16GB
Memory Usage:
Allocated: 14.9 GB
Cached:    15.1 GB
